In [90]:
from typing import Callable
from dataclasses import dataclass
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import root, approx_fprime
from scipy import linalg as la

# Implicit Runge-Kutta method

In [34]:
def RungeKuttaImplicit(f: Callable[[NDArray], NDArray], x0: NDArray, dt: float) -> NDArray:
    N = x0.size

    b1: float = 0.5
    b2: float = 0.5

    a11 = 0.25
    a12 = 0.25 - np.sqrt(3) / 6.0
    a21 = 0.25 + np.sqrt(3) / 6.0
    a22 = 0.25

    k1_res = lambda k1, k2: f(x0 + a11 * k1 * dt + a12 * k2 * dt) - k1
    k2_res = lambda k1, k2: f(x0 + a21 * k1 * dt + a22 * k2 * dt) - k2

    k1_0 = f(x0)
    k2_0 = f(x0)

    ks_0 = np.zeros(2 * N)
    ks_0[0:N] = k1_0
    ks_0[N:] = k2_0

    def residual(ks: NDArray) -> NDArray:
        res_vals = ks.copy()
        res_vals[0:N] = k1_res(ks[:N], ks[N:])
        res_vals[N:] = k2_res(ks[:N], ks[N:])

        return res_vals
    
    ks = root(residual, ks_0).x

    k1 = ks[0:N]
    k2 = ks[N:]

    return x0 + dt * (b1 * k1 + b2 * k2)

# Define Chemical System

In [35]:
@dataclass
class ChemicalState:
    conc: NDArray
    tot_conc: NDArray

@dataclass
class TotalConstants:
    tot_mat: NDArray

@dataclass
class KineticConstants:
    kin_mat: NDArray
    kin_rate: NDArray
    eq_const: NDArray

@dataclass
class EquilibriumConstants:
    eq_consts: NDArray
    stoich_mat: NDArray

# Solving Equilibrium

In [68]:
def solve_equilibrium(chem: ChemicalState, eq: EquilibriumConstants, tot: TotalConstants) -> ChemicalState:
    N: int = chem.tot_conc.size
    x0: NDArray = np.full(N, -1)

    log_x_p: NDArray = la.pinv(eq.stoich_mat) @ eq.eq_consts
    stoich_null: NDArray = la.null_space(eq.stoich_mat)

    def log_c(log_x: NDArray) -> NDArray:
        return log_x_p + stoich_null @ log_x
    
    def mass_err(log_x: NDArray) -> NDArray:
        c: NDArray = np.exp(log_c(log_x))
        return tot.tot_mat @ c - chem.tot_conc

    concVec = np.exp(log_c(root(mass_err, x0).x))
    
    return ChemicalState(concVec, chem.tot_conc)

# Kinetic Rates

In [197]:
def kinetic_rate(tot_conc: NDArray, surface_area: NDArray, eq: EquilibriumConstants, kin: KineticConstants, tot: TotalConstants) -> NDArray:
    c0 = np.zeros(eq.stoich_mat.shape[1])
    chem = ChemicalState(c0, tot_conc)
    chem_new = solve_equilibrium(chem, eq, tot)
    conc: NDArray = chem_new.conc

    log_c = np.log(conc)
    log_iap = kin.kin_mat @ log_c
    iap = np.exp(log_iap)
    min_rates = surface_area * (10.0 ** kin.kin_rate) * (1.0 - iap / (kin.eq_const ** 10.0))
    # min_rates = surface_area * (10.0 ** kin.kin_rate) * (1.0 - log_iap)
    min_rate_mat = np.diag(min_rates)
    component_mat = min_rate_mat @ kin.kin_mat
    spec_rate = component_mat.T @ np.ones(kin.kin_mat.shape[0])

    return tot.tot_mat @ spec_rate

def solve_kinetic(chem: ChemicalState, surface_area: NDArray, eq: EquilibriumConstants, kin: KineticConstants, tot: TotalConstants, dt: float) -> ChemicalState:
    def get_kin_rate(tot_conc: NDArray) -> NDArray:
        return kinetic_rate(tot_conc, surface_area, eq, kin, tot)
    
    new_tot_conc: NDArray = RungeKuttaImplicit(get_kin_rate, chem.tot_conc, dt)
    chem = ChemicalState(np.zeros(1), new_tot_conc)

    return solve_equilibrium(chem, eq, tot)

In [198]:
dt: float = 500.0
kin_rate: NDArray = np.array([-9.19])
kin_eq_consts: NDArray = np.array([-7.3])
kin_mat: NDArray = np.array([[1, -1, 1, 0, 0]], dtype=float)
eq_consts: NDArray = np.array([9.617, -6.345])
stoich_mat: NDArray = np.array([
    [0, 1, -1, 1, 0],
    [0, -1, -1, 0, 1]], dtype=float)

tot_mat: NDArray = np.array([
    [1, 0, 0, 0, 0],
    [2, 1, -1, -2, 0],
    [0, 0, 1, 1, 1]], dtype=float)

conc_0: NDArray = np.ones(5)
tot_0: NDArray = np.array([1e-3, 0.0, 0.002])
surf_area: NDArray = np.array([1.0])

eq = EquilibriumConstants(eq_consts=eq_consts, stoich_mat=stoich_mat)
kin = KineticConstants(kin_mat=kin_mat, kin_rate=kin_rate, eq_const=kin_eq_consts)
tot = TotalConstants(tot_mat=tot_mat)
chem = ChemicalState(conc_0, tot_0)

In [199]:
solve_kinetic(chem, surf_area, eq, kin, tot, dt)

ChemicalState(conc=array([1.00032283e-03, 1.99999973e-03, 2.66391239e-10, 2.00032256e-03,
       9.35300978e-16]), tot_conc=array([0.00100032, 0.        , 0.00200032]))

In [203]:
kinetic_rate(chem.tot_conc, surf_area, eq, kin, tot)

array([6.45654229e-10, 0.00000000e+00, 6.45654229e-10])